In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sanskrit-english-dataset/combined.csv


In [ ]:
import numpy as np           # NumPy for numerical computations 
import pandas as pd          # Pandas is a data manipulation and analysis 
import matplotlib.pyplot as plt  # Data visualizations.
import seaborn as sns        # Data visualization r
import tensorflow as tf      # Open-source machine learning framework 
import plotly.express as px  # Interactive and expressive plots- data viz
import tensorflow_datasets as tfds  # For downloading and preparing machine learning datasets.
from tensorflow import keras  # Open-source deep learning framework for neural network
from keras import layers     # Keras layers for building neural networks.
from colorama import Fore, Style  # Colorama is used for colored terminal text
from IPython.core.display import HTML  # IPython provides enhanced interactive programming in Python.

from tensorflow.keras.layers import Input, Embedding, LSTM, Dense  # Additional Keras layers for building neural network architectures.

from tensorflow.keras import backend as K # For low-level operations and utilities for TensorFlow and Keras.
from tensorflow.random import set_seed # Set a seed for random number generators.


In [ ]:
from colorama import Fore, Style
from IPython.core.display import HTML

K = keras.backend
ON_KAGGLE = os.getenv("KAGGLE_KERNEL_RUN_TYPE") is not None
FONT_COLOR = "#141B4D"
BACKGROUND_COLOR = "#F6F5F5"
CLR = (Style.BRIGHT + Fore.BLACK) if ON_KAGGLE else (Style.BRIGHT + Fore.WHITE)
RED = Style.BRIGHT + Fore.RED
BLUE = Style.BRIGHT + Fore.BLUE
CYAN = Style.BRIGHT + Fore.CYAN
RESET = Style.RESET_ALL
NOTEBOOK_PALETTE = {
    "DeepPlum": "#3F384A",
    "RubyRed": "#E04C5F",
    "SunburstOrange": "#FFB74D",
}


In [ ]:
df = "/kaggle/input/sanskrit-english-dataset/combined.csv"
df = pd.read_csv(df, encoding="utf-8", engine="pyarrow")
df = df.sample(len(df), random_state=42)
df.head()

In [5]:
# List of columns to drop
columns_to_drop = ['book', 'volume', 'parva', 'cnum', 'chapter', 'description', 'shloka']

# Drop the specified columns
df = df.drop(columns_to_drop, axis=1)


In [6]:
df.head()

,sn,en
44195,स राजा महतीं सेनां योजयित्वा षडङ्गिनीम्। अमृष्...,"The king, having collected a large army consis..."
47393,वैशम्पायन उवाच पितुर्वचो निशम्यैतद् धृतराष्ट्र...,Dhritarashtra said Having heard the words of f...
20718,आवृण्वानं महाज्वालमर्चिभिः सर्वतोऽम्बरम्। दहन्...,The terrible flames of that fire covered whole...
24340,पाण्डवाश्च महात्मानः प्रतिसंदिश्य वै कवेः। गङ्...,"The illustrious Pandavas also, sending some me..."
34011,बडिशोऽयं त्वया ग्रस्तः कालसूत्रेण लम्बितः। मत्...,"(By thus allowing you to commit this act), you..."


In [12]:
# Rearranging the columns

df = df[['sn','en']]
df.head()

,sn,en
44195,स राजा महतीं सेनां योजयित्वा षडङ्गिनीम्। अमृष्...,"The king, having collected a large army consis..."
47393,वैशम्पायन उवाच पितुर्वचो निशम्यैतद् धृतराष्ट्र...,Dhritarashtra said Having heard the words of f...
20718,आवृण्वानं महाज्वालमर्चिभिः सर्वतोऽम्बरम्। दहन्...,The terrible flames of that fire covered whole...
24340,पाण्डवाश्च महात्मानः प्रतिसंदिश्य वै कवेः। गङ्...,"The illustrious Pandavas also, sending some me..."
34011,बडिशोऽयं त्वया ग्रस्तः कालसूत्रेण लम्बितः। मत्...,"(By thus allowing you to commit this act), you..."


In [9]:
# Choose 5% of the data randomly
percentage_to_select = 0.05
sdf = df.sample(frac=percentage_to_select, random_state=42)

# Verify the shape of the selected_easy_dataset
print("Selected Data Shape:", sdf.shape)

Selected Data Shape: (4652, 2)


In [14]:

# Pull the English and Sanskrit sentences from the DataFrame
sentences_en = sdf["en"].to_numpy()
sentences_sn = sdf["sn"].to_numpy()

# set to 0.1, indicating that 10% of the data will be used for validation
valid_fraction = 0.1

# Calculating the number of data points to be used for validation
valid_len = int(valid_fraction * len(sdf))

# Creating the training and validation set

sentences_sn_train = sentences_sn[:-valid_len]  # Training data for English sentences
# Here `-valid_len` for  selecting all data points except the last valid_len data points

sentences_en_train = sentences_en[:-valid_len]  # Training data for Sanskrit sentences


sentences_sn_valid = sentences_sn[-valid_len:]  # Validation data for English sentences


sentences_en_valid = sentences_en[-valid_len:]  # Validation data for Sanskrit sentences


In [15]:
def prepare_input_and_target(sentences_sn, sentences_en):
    """Return data in the format: `((encoder_input, decoder_input), target)`"""
    return (sentences_sn, b"startofseq " + sentences_en), sentences_en + b" endofseq"


def from_sentences_dataset(
    sentences_sn,
    sentences_en,
    batch_size=32,
    cache=True,
    shuffle=False,
    shuffle_buffer_size=10_000,
    seed=None,
):
    """Creates `TensorFlow` dataset for encoder-decoder RNN from given sentences."""
    dataset = tf.data.Dataset.from_tensor_slices((sentences_sn, sentences_en))
    dataset = dataset.map(prepare_input_and_target, num_parallel_calls=tf.data.AUTOTUNE)
    if cache:
        dataset = dataset.cache()
    if shuffle:
        dataset = dataset.shuffle(shuffle_buffer_size, seed=seed)
    return dataset.batch(batch_size)

In [16]:
benchmark_ds = from_sentences_dataset(sentences_sn_train, sentences_en_train)
benchmark_ds = benchmark_ds.prefetch(tf.data.AUTOTUNE)
bench_results = tfds.benchmark(benchmark_ds, batch_size=32)


************ Summary ************



  0%|          | 0/131 [00:00<?, ?it/s]

Examples/sec (First included) 16200.86 ex/sec (total: 4224 ex, 0.26 sec)
Examples/sec (First only) 292.49 ex/sec (total: 32 ex, 0.11 sec)
Examples/sec (First excluded) 27702.37 ex/sec (total: 4192 ex, 0.15 sec)


In [17]:
example_ds = from_sentences_dataset(
    sentences_sn_train, sentences_en_train, batch_size=4
)
list(example_ds.take(1))[0]

((<tf.Tensor: shape=(4,), dtype=string, numpy=
  array([b'\xe0\xa4\xa4\xe0\xa4\xa4\xe0\xa5\x8b \xe0\xa4\xa6\xe0\xa4\x97\xe0\xa5\x8d\xe0\xa4\xa7\xe0\xa4\xbe\xe0\xa4\xb5\xe0\xa4\xbf\xe0\xa4\xae\xe0\xa5\x8c \xe0\xa4\xaa\xe0\xa4\x95\xe0\xa5\x8d\xe0\xa4\xb7\xe0\xa5\x8c \xe0\xa4\xa8 \xe0\xa4\xa6\xe0\xa4\x97\xe0\xa5\x8d\xe0\xa4\xa7\xe0\xa5\x8c \xe0\xa4\xa4\xe0\xa5\x81 \xe0\xa4\x9c\xe0\xa4\x9f\xe0\xa4\xbe\xe0\xa4\xaf\xe0\xa5\x81\xe0\xa4\xb7\xe0\xa4\x83\xe0\xa5\xa4 \xe0\xa4\xa4\xe0\xa4\xa6\xe0\xa4\xbe \xe0\xa4\xae\xe0\xa5\x87 \xe0\xa4\x9a\xe0\xa4\xbf\xe0\xa4\xb0\xe0\xa4\xa6\xe0\xa5\x83\xe0\xa4\xb7\xe0\xa5\x8d\xe0\xa4\x9f\xe0\xa4\x83 \xe0\xa4\xb8 \xe0\xa4\xad\xe0\xa5\x8d\xe0\xa4\xb0\xe0\xa4\xbe\xe0\xa4\xa4\xe0\xa4\xbe \xe0\xa4\x97\xe0\xa5\x83\xe0\xa4\xa7\xe0\xa5\x8d\xe0\xa4\xb0\xe0\xa4\xaa\xe0\xa4\xa4\xe0\xa4\xbf\xe0\xa4\x83 \xe0\xa4\xaa\xe0\xa5\x8d\xe0\xa4\xb0\xe0\xa4\xbf\xe0\xa4\xaf\xe0\xa4\x83\xe0\xa5\xa5 \xe0\xa4\xa8\xe0\xa4\xbf\xe0\xa4\xb0\xe0\xa5\x8d\xe0\xa4\xa6\xe0\xa4\x97\xe0\xa5\x8d\xe0

In [18]:
example_ds.cardinality()  # Number of batches per epoch.

<tf.Tensor: shape=(), dtype=int64, numpy=1047>

In [19]:
class ColoramaVerbose(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(
            f"{CLR}Epoch: {RED}{epoch + 1:02d}{CLR} -",
            f"{CLR}loss: {RED}{logs['loss']:.5f}{CLR} -",
            f"{CLR}accuracy: {RED}{logs['accuracy']:.5f}{CLR} -",
            f"{CLR}val_loss: {RED}{logs['val_loss']:.5f}{CLR} -",
            f"{CLR}val_accuracy: {RED}{logs['val_accuracy']:.5f}",
        )

In [21]:

# Checking the data structure of the sentences
first_element = next(iter(example_ds))  # Retrieve the first element in the dataset.

# Access sentences[0] within the tuple.
sentences_0_data = first_element[1][1]
sentences_1_data = first_element[0][0]

# Display the contents of sentences.
print("English \n", sentences_0_data)
print()
print("Sanskrit \n",sentences_1_data)


English 
 tf.Tensor(b'Bhishma said Malevolent persons always commit sinful acts and feel an irresistible inclination for doing them. They speak ill of others and are themselves censured. They always consider themselves as cheated of what is their due. endofseq', shape=(), dtype=string)

Sanskrit 
 tf.Tensor(
[b'\xe0\xa4\xa4\xe0\xa4\xa4\xe0\xa5\x8b \xe0\xa4\xa6\xe0\xa4\x97\xe0\xa5\x8d\xe0\xa4\xa7\xe0\xa4\xbe\xe0\xa4\xb5\xe0\xa4\xbf\xe0\xa4\xae\xe0\xa5\x8c \xe0\xa4\xaa\xe0\xa4\x95\xe0\xa5\x8d\xe0\xa4\xb7\xe0\xa5\x8c \xe0\xa4\xa8 \xe0\xa4\xa6\xe0\xa4\x97\xe0\xa5\x8d\xe0\xa4\xa7\xe0\xa5\x8c \xe0\xa4\xa4\xe0\xa5\x81 \xe0\xa4\x9c\xe0\xa4\x9f\xe0\xa4\xbe\xe0\xa4\xaf\xe0\xa5\x81\xe0\xa4\xb7\xe0\xa4\x83\xe0\xa5\xa4 \xe0\xa4\xa4\xe0\xa4\xa6\xe0\xa4\xbe \xe0\xa4\xae\xe0\xa5\x87 \xe0\xa4\x9a\xe0\xa4\xbf\xe0\xa4\xb0\xe0\xa4\xa6\xe0\xa5\x83\xe0\xa4\xb7\xe0\xa5\x8d\xe0\xa4\x9f\xe0\xa4\x83 \xe0\xa4\xb8 \xe0\xa4\xad\xe0\xa5\x8d\xe0\xa4\xb0\xe0\xa4\xbe\xe0\xa4\xa4\xe0\xa4\xbe \xe0\xa4\x97\xe0\xa5\x83\xe

In [28]:
def adapt_compile_and_fit(
    model,
    train_dataset,
    valid_dataset,
    n_epochs=25,
    n_patience=5,
    init_lr=0.001,
    lr_decay_rate=0.1,
    colorama_verbose=False,
):
    """Takes the model vectorization layers and adapts them to the training data.
    Then, it prepares the final datasets vectorizing targets and prefetching,
    and finally trains the given model. Additionally, provides learning rate scheduling
    (exponential decay), early stopping and colorama verbose."""

    model.vectorization_sn.adapt(
        train_dataset.map(
            lambda sentences, target: sentences[1],  # Sanskrit sentences.
            num_parallel_calls=tf.data.AUTOTUNE,
        )
    )
    model.vectorization_en.adapt(
        train_dataset.map(
            lambda sentences, target: sentences[0] + b" endofseq",  # English sentences.
            num_parallel_calls=tf.data.AUTOTUNE,
        )
    )

    train_dataset_prepared = train_dataset.map(
        lambda sentences, target: (sentences, model.vectorization_en(target)),
        num_parallel_calls=tf.data.AUTOTUNE,
    ).prefetch(tf.data.AUTOTUNE)

    valid_dataset_prepared = valid_dataset.map(
        lambda sentences, target: (sentences, model.vectorization_en(target)),
        num_parallel_calls=tf.data.AUTOTUNE,
    ).prefetch(tf.data.AUTOTUNE)

    early_stopping_cb = keras.callbacks.EarlyStopping(
        monitor="val_accuracy", patience=n_patience, restore_best_weights=True
    )
    
    # The line below doesn't work with multi-file interleaving.
    # n_decay_steps = n_epochs * train_dataset_prepared.cardinality().numpy()
    # Less elegant solution.
    n_decay_steps = n_epochs * len(list(train_dataset_prepared))
    scheduled_lr = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=init_lr,
        decay_steps=n_decay_steps,
        decay_rate=lr_decay_rate,
    )

    model_callbacks = [early_stopping_cb]
    verbose_level = 1
    if colorama_verbose:
        model_callbacks.append(ColoramaVerbose())
        verbose_level = 0

    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=keras.optimizers.RMSprop(learning_rate=scheduled_lr),
        metrics=["accuracy"],
    )

    return model.fit(
        train_dataset_prepared,
        epochs=n_epochs,
        validation_data=valid_dataset_prepared,
        callbacks=model_callbacks,
        verbose=verbose_level,
    )

In [44]:
def translate(model, sentence_sn):
    translation = ""
    for word_idx in range(model.max_sentence_len):
        X_encoder = np.array([sentence_sn])
        X_decoder = np.array(["startofseq " + translation])
        # Last token's probas.
        y_proba = model.predict((X_encoder, X_decoder), verbose=0)[0, word_idx]
        predicted_word_id = np.argmax(y_proba)
        predicted_word = model.vectorization_en.get_vocabulary()[predicted_word_id]
        if predicted_word == "endofseq":
            break
        translation += " " + predicted_word
    return translation.strip()

In [45]:
class PositionalEncoding(layers.Layer):
    def __init__(
        self, max_sentence_len=50, embedding_size=256, dtype=tf.float32, **kwargs
    ):
        super().__init__(dtype=dtype, **kwargs)
        if not embedding_size % 2 == 0:
            raise ValueError("The `embedding_size` must be even.")

        p, i = np.meshgrid(np.arange(max_sentence_len), np.arange(embedding_size // 2))
        pos_emb = np.empty((1, max_sentence_len, embedding_size))
        pos_emb[:, :, 0::2] = np.sin(p / 10_000 ** (2 * i / embedding_size)).T
        pos_emb[:, :, 1::2] = np.cos(p / 10_000 ** (2 * i / embedding_size)).T
        self.positional_embedding = tf.constant(pos_emb.astype(self.dtype))
        self.supports_masking = True

    def call(self, inputs):
        batch_max_length = tf.shape(inputs)[1]
        return inputs + self.positional_embedding[:, :batch_max_length]

In [46]:
class Encoder(layers.Layer):
    def __init__(
        self,
        embedding_size=256,
        n_attention_heads=8,
        n_units_dense=256,
        dropout_rate=0.2,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.multi_head_attention = layers.MultiHeadAttention(
            n_attention_heads, embedding_size, dropout=dropout_rate
        )
        self.feed_forward = keras.Sequential(
            [
                layers.Dense(
                    n_units_dense, activation="relu", kernel_initializer="he_normal"
                ),
                layers.Dense(embedding_size, kernel_initializer="he_normal"),
                layers.Dropout(dropout_rate),
            ]
        )
        self.add = layers.Add()
        self.normalization = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        Z = inputs
        skip_Z = Z
        Z = self.multi_head_attention(Z, value=Z, attention_mask=mask)
        Z = self.normalization(self.add([Z, skip_Z]))
        skip_Z = Z
        Z = self.feed_forward(Z)
        return self.normalization(self.add([Z, skip_Z]))


class Decoder(layers.Layer):
    def __init__(
        self,
        embedding_size=256,
        n_attention_heads=8,
        n_units_dense=256,
        dropout_rate=0.2,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.masked_multi_head_attention = layers.MultiHeadAttention(
            n_attention_heads, embedding_size, dropout=dropout_rate
        )
        self.multi_head_attention = layers.MultiHeadAttention(
            n_attention_heads, embedding_size, dropout=dropout_rate
        )
        self.feed_forward = keras.Sequential(
            [
                layers.Dense(
                    n_units_dense, activation="relu", kernel_initializer="he_normal"
                ),
                layers.Dense(embedding_size, kernel_initializer="he_normal"),
                layers.Dropout(dropout_rate),
            ]
        )
        self.add = layers.Add()
        self.normalization = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        decoder_mask, encoder_mask = mask  # type: ignore
        Z, encoder_output = inputs
        Z_skip = Z
        Z = self.masked_multi_head_attention(Z, value=Z, attention_mask=decoder_mask)
        Z = self.normalization(self.add([Z, Z_skip]))
        Z_skip = Z
        Z = self.multi_head_attention(
            Z, value=encoder_output, attention_mask=encoder_mask
        )
        Z = self.normalization(self.add([Z, Z_skip]))
        Z_skip = Z
        Z = self.feed_forward(Z)
        return self.normalization(self.add([Z, Z_skip]))

In [47]:
class Transformer(keras.Model):
    def __init__(
        self,
        vocabulary_size=5000,
        max_sentence_len=50,
        embedding_size=256,
        n_encoder_decoder_blocks=1,
        n_attention_heads=8,
        n_units_dense=256,
        dropout_rate=0.2,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.max_sentence_len = max_sentence_len

        self.vectorization_sn = layers.TextVectorization(
            vocabulary_size, output_sequence_length=max_sentence_len
        )
        self.vectorization_en = layers.TextVectorization(
            vocabulary_size, output_sequence_length=max_sentence_len
        )
        self.encoder_embedding = layers.Embedding(
            vocabulary_size, embedding_size, mask_zero=True
        )
        self.decoder_embedding = layers.Embedding(
            vocabulary_size, embedding_size, mask_zero=True
        )
        self.positional_encoding = PositionalEncoding(max_sentence_len, embedding_size)
        self.encoder_blocks = [
            Encoder(embedding_size, n_attention_heads, n_units_dense, dropout_rate)
            for _ in range(n_encoder_decoder_blocks)
        ]
        self.decoder_blocks = [
            Decoder(embedding_size, n_attention_heads, n_units_dense, dropout_rate)
            for _ in range(n_encoder_decoder_blocks)
        ]
        self.output_layer = layers.Dense(vocabulary_size, activation="softmax")

    def call(self, inputs):
        encoder_inputs, decoder_inputs = inputs

        encoder_input_ids = self.vectorization_sn(encoder_inputs)
        decoder_input_ids = self.vectorization_en(decoder_inputs)

        encoder_embeddings = self.encoder_embedding(encoder_input_ids)
        decoder_embeddings = self.decoder_embedding(decoder_input_ids)

        encoder_pos_embeddings = self.positional_encoding(encoder_embeddings)
        decoder_pos_embeddings = self.positional_encoding(decoder_embeddings)

        encoder_pad_mask = tf.math.not_equal(encoder_input_ids, 0)[:, tf.newaxis]
        decoder_pad_mask = tf.math.not_equal(decoder_input_ids, 0)[:, tf.newaxis]

        # From original paper: "This masking, combined with fact that the output
        # embeddings are offset by one position, ensures that the predictions for
        # position i can depend only on the known outputs at positions less than i."
        batch_max_len_decoder = tf.shape(decoder_embeddings)[1]
        decoder_causal_mask = tf.linalg.band_part(  # Lower triangular matrix.
            tf.ones((batch_max_len_decoder, batch_max_len_decoder), tf.bool), -1, 0
        )
        decoder_mask = decoder_causal_mask & decoder_pad_mask

        Z = encoder_pos_embeddings
        for encoder_block in self.encoder_blocks:
            Z = encoder_block(Z, mask=encoder_pad_mask)

        encoder_output = Z 
        Z = decoder_pos_embeddings
        for decoder_block in self.decoder_blocks:
            Z = decoder_block(
                [Z, encoder_output], mask=[decoder_mask, encoder_pad_mask]
            )

        return self.output_layer(Z)

In [39]:
K.clear_session()
tf.random.set_seed(42)

transformer = Transformer(max_sentence_len=50)
transformer_history = adapt_compile_and_fit(
    transformer, easy_train_ds, easy_valid_ds, colorama_verbose=True
)

Epoch: 01 - loss: 0.30007 - accuracy: 0.99154 - val_loss: 0.00747 - val_accuracy: 0.99914
Epoch: 02 - loss: 0.00702 - accuracy: 0.99919 - val_loss: 0.00714 - val_accuracy: 0.99914
Epoch: 03 - loss: 0.00676 - accuracy: 0.99919 - val_loss: 0.00704 - val_accuracy: 0.99914
Epoch: 04 - loss: 0.00673 - accuracy: 0.99919 - val_loss: 0.00692 - val_accuracy: 0.99914
Epoch: 05 - loss: 0.00673 - accuracy: 0.99919 - val_loss: 0.00697 - val_accuracy: 0.99914
Epoch: 06 - loss: 0.00670 - accuracy: 0.99919 - val_loss: 0.00684 - val_accuracy: 0.99914


In [40]:
fig = px.line(
    transformer_history.history,
    markers=True,
    height=540,
    width=840,
    symbol="variable",
    labels={"variable": "Variable", "value": "Value", "index": "Epoch"},
    title="Easy Dataset - Transformer Training Process",
    color_discrete_sequence=px.colors.diverging.balance_r,
)
fig.update_layout(
    font_color=FONT_COLOR,
    title_font_size=18,
    plot_bgcolor=BACKGROUND_COLOR,
    paper_bgcolor=BACKGROUND_COLOR,
)
fig.show()

In [58]:
# Create a DataFrame from the history dictionary
tdf = pd.DataFrame(history)

# Calculate the mean (average) for each column
average_values = tdf.mean()

# Format the average values with two decimal places
average_values = average_values.round(4)

# Print the average values
print(average_values)

loss            0.0557
accuracy        0.9979
val_loss        0.0071
val_accuracy    0.9991
dtype: float64


In [59]:
translation1 = translate(transformer,"In consequence of which my wings were burnt but those of Jatayu were not.")



print(CLR + "Model Translations:")
print(BLUE + "In consequence of which my wings were burnt but those of Jatayu were not.".ljust(25), RED + "-> ", BLUE + translation1)

Model Translations:
In consequence of which my wings were burnt but those of Jatayu were not. ->  [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK]
